БИБЛИОТЕКИ

In [1]:
import pandas as pd
import numpy as np

In [2]:
# монтируем свой диск...
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


ИМЕНА ФАЙЛОВ ВХОДА/ВЫХОДА

In [3]:
# Входной обучающий датасет
train_dataset_file = '/content/drive/My Drive/Bases/DTP/train_dataset_train.csv'

# Входной тестовый датасет
test_dataset_file = '/content/drive/My Drive/Bases/DTP/test_dataset_test.csv'

# Выходной файл тестовой выборки
#x_test_file = '/content/drive/My Drive/Bases/DTP/x_test.dat'

# Выходные файлы обучающей выборки для Шага №1
#x_train_0_file = '/content/drive/My Drive/Bases/DTP/x_train_0.dat'
#y_train_0_file = '/content/drive/My Drive/Bases/DTP/y_train_0.dat'

# Выходные файлы обучающей выборки для Шага №2
#x_train_1_file = '/content/drive/My Drive/Bases/DTP/x_train_1.dat'
#y_train_1_file = '/content/drive/My Drive/Bases/DTP/y_train_1.dat'

# Выходные файлы обучающей выборки для Шага №3
x_train_2_file = '/content/drive/My Drive/Bases/DTP/x_train_22.dat'
y_train_2_file = '/content/drive/My Drive/Bases/DTP/y_train_22.dat'


ЗАГРУЗКА ДАТАСЕТОВ И ОБЪЕДИНЕНИЕ ИХ В ЕДИНУЮ ТАБЛИЦУ

In [4]:
# загружаем обучающий датасет...
df = pd.read_csv(train_dataset_file)

# фиксируем размер обучающего датасета 
train_base_size = df.shape[0]

# загружаем тестовый датасет...
df_test = pd.read_csv(test_dataset_file)

# объединяем датасеты в единую таблицу
for i in range(df_test.shape[0]):
  rab = {}
  rab['id'] = df_test['id'][i]
  rab['Дата'] = df_test['Дата'][i]
  rab['Время'] = df_test['Время'][i]
  rab['Место'] = df_test['Место'][i]
  rab['Улица'] = df_test['Улица'][i]
  rab['Дом'] = df_test['Дом'][i]
  rab['Дорога'] = df_test['Дорога'][i]
  rab['Километр'] = df_test['Километр'][i]
  rab['Метр'] = df_test['Метр'][i]
  rab['Вид ДТП'] = df_test['Вид ДТП'][i]
  rab['Погибло'] = 0
  rab['Погибло детей'] = 0
  rab['Ранено'] = 0
  rab['Ранено детей'] = 0
  df = df.append(rab, ignore_index=True)

ФУНКЦИИ ДЛЯ РАЗБИВКИ СТОЛБЦА 'МЕСТО' НА ТРИ ЧАСТИ

In [5]:
# извлечение первой составляющей из столбца 'Место' 
def get_place_a(val):

  rab = val
  
  pos = rab.find(',')
  if pos == -1:
    return '0'
  else:
    rab = rab[pos+2:]
  pos = rab.find(',')
  if pos > -1:
    rab = rab[:pos]

  return rab

# извлечение второй составляющей из столбца ('Место') 
def get_place_b(val):
  rab = val
  
  pos = rab.find(',')
  if pos == -1:
    return '0'
  else:
    rab = rab[pos+2:]
  
  pos = rab.find(',')
  if pos == -1:
    return '0'
  else:
    rab = rab[pos+2:]
  pos = rab.find(',')
  if pos > -1:
    rab = rab[:pos]

  return rab  

# извлечение третьей составляющей из столбца ('Место') 
def get_place_c(val):
  rab = val
  
  pos = rab.find(',')
  if pos == -1:
    return '0'
  else:
    rab = rab[pos+2:]
  
  pos = rab.find(',')
  if pos == -1:
    return '0'
  else:
    rab = rab[pos+2:]

  pos = rab.find(',')
  if pos == -1:
    return '0'
  else:
    rab = rab[pos+2:]
 
  return rab   

ОБРАБОТКА ТАБЛИЦЫ

In [6]:
# пропуски заполняем 0
df = df.fillna('0') 

# извлекаем месяц
df['Месяц'] = df['Дата'].str[3:5].astype(int)

# извлекаем час...
df['Час'] = df['Время'].str[:2].astype(int)

# удаляем столбцы "Дата" и "Время"
del df['Дата']
del df['Время']

# раскладываем место на три части...
df['Место_1'] = [get_place_a(x) for x in df['Место']] 
df['Место_2'] = [get_place_b(x) for x in df['Место']] 
df['Место_3'] = [get_place_c(x) for x in df['Место']] 
# удаляем столбец "Место"
del df['Место']

# приводим Метры и Километры в int
df['Километр'] = df['Километр'].astype('int')
df['Метр'] = df['Метр'].astype('int')

# добавляем столбец 'Место_4' (на основе слияния Улица-Дом либо Дорога-Километр-Метр)

val = []

for i in range(df.shape[0]):

  if df['Улица'][i] == '0':
    val.append(df['Дорога'][i] + df['Километр'][i].astype('str'))
  else:
    val.append(df['Улица'][i] + df['Дом'][i])

df['Место_4'] = val

# Переводим нужные столбцы в категории

df["Вид ДТП"] = pd.Categorical(df["Вид ДТП"])
df["Вид ДТП"].astype('category').cat.codes
df["Вид ДТП"] = df["Вид ДТП"].cat.codes

df["Место_1"] = pd.Categorical(df["Место_1"])
df["Место_1"].astype('category').cat.codes
df["Место_1"] = df["Место_1"].cat.codes

df["Место_2"] = pd.Categorical(df["Место_2"])
df["Место_2"].astype('category').cat.codes
df["Место_2"] = df["Место_2"].cat.codes

df["Место_3"] = pd.Categorical(df["Место_3"])
df["Место_3"].astype('category').cat.codes
df["Место_3"] = df["Место_3"].cat.codes

df["Место_4"] = pd.Categorical(df["Место_4"])
df["Место_4"].astype('category').cat.codes
df["Место_4"] = df["Место_4"].cat.codes

df["Месяц"] = pd.Categorical(df["Месяц"])
df["Месяц"].astype('category').cat.codes
df["Месяц"] = df["Месяц"].cat.codes

df["Час"] = pd.Categorical(df["Час"])
df["Час"].astype('category').cat.codes
df["Час"] = df["Час"].cat.codes

# удаляем ненужные столбцы
del df['Улица'], df['Дом'], df['Дорога'], df['Километр'], df['Метр']

ДОБАВЛЯЕМ СТОЛБЕЦ 'КЛАСС' С СИМВОЛАМИ В СООТВЕТСВИИ С ПРОИСШЕСТВИЯМИ

In [7]:
df['Класс'] = (df['Погибло'].astype('str') + 
               df['Погибло детей'].astype('str') + 
               df['Ранено'].astype('str') + 
               df['Ранено детей'].astype('str'))

# смотрим на распределение данных по классам
df['Класс'].value_counts()

0000    37781
0010     2918
0020      563
0011      506
1000      211
0030      141
0021       91
0022       43
0031       38
1010       38
0040       30
0041       27
1020       24
0032       19
2000       18
0050       17
0042       15
1030        7
0033        7
3000        6
2010        6
0052        4
3061        3
0084        3
0062        3
1110        3
2070        3
0060        3
2020        3
2120        3
0054        3
1031        2
1041        2
1040        2
2030        2
0051        2
3110        2
1021        2
1051        2
1022        2
1121        2
1100        1
1131        1
Name: Класс, dtype: int64

In [10]:
# список классов
classes_list = list(df['Класс'].unique())
# удаляем ненужные классы
classes_list.remove('0000')
classes_list.remove('0010')

print(classes_list)

['0020', '0011', '1010', '0050', '0041', '0030', '1000', '0031', '0021', '0032', '1022', '0022', '0040', '1030', '1131', '0062', '0060', '2000', '0052', '2030', '1020', '1051', '1110', '0042', '1021', '2010', '0033', '3110', '3061', '0084', '3000', '0054', '0051', '2020', '1041', '1031', '2070', '1100', '2120', '1040', '1121']


СОСТАВЛЕНИЕ КОРЗИН ПО КЛАССАМ

In [15]:
# словарь корзин
my_dict = {} 

# обнуление списков для X и Y
for i in range(len(classes_list)):
  my_dict[i] = [[], []]

# проходим по всем записям train без тестовой выборки
for i in range(train_base_size):

  if df['Класс'][i] != '0000' and df['Класс'][i] != '0010':

    x_vec = []
  
    x_vec.append(df['Вид ДТП'][i].astype('float')) 

    x_vec.append(df['Место_1'][i].astype('float')) 
   
    x_vec.append(df['Место_2'][i].astype('float')) 

    x_vec.append(df['Место_3'][i].astype('float')) 

    x_vec.append(df['Место_4'][i].astype('float')) 
    
    x_vec.append(df['Месяц'][i].astype('float')) 

    x_vec.append(df['Час'][i].astype('float')) 

    # добавление записи в корзину словаря
    my_dict[classes_list.index(df['Класс'][i])][0].append(x_vec)
    my_dict[classes_list.index(df['Класс'][i])][1].append([df['Погибло'][i].astype('float'),
                                                           df['Погибло детей'][i],
                                                           df['Ранено'][i].astype('float'),
                                                           df['Ранено детей'][i].astype('float')])

С КАЖДОЙ КОРЗИНЫ БЕРЕМ ПО 5% НА ВАЛИДАЦИОННЫЙ ДАТАСЕТ 

In [16]:
x_samples = []
y_samples = []

for i in range(len(classes_list)):

  n = len(my_dict[i][0]) # элементов в корзине
  
  k = round(n/20) # столько элементов берется из корзины

  for j in range(k):

    x_samples.append(my_dict[i][0][j])
    y_samples.append(my_dict[i][1][j])
  
  # взятые первые 5% элементов удаляются из корзины

  my_dict[i][0] = my_dict[i][0][k:]
  my_dict[i][1] = my_dict[i][1][k:]

# результирующая валидационная выборка
x_val = np.array(x_samples)
y_val = np.array(y_samples)

СБОРКА ТРЕНЕРОВОЧНОЙ ВЫБОРКИ ИЗ КОРЗИН ДЛЯ КАЖДОГО КЛАССА НАКИДЫВАЕМ СТОЛЬКО ЖЕ, СКОЛЬКО В ЗАПИСЕЙ В НУЛЕВОЙ КОРЗИНЕ

In [17]:
x_samples = []
y_samples = []

# размер первой корзины
num = len(my_dict[0][0])

for k in range(num):

  for i in range(len(my_dict)):

    n = len(my_dict[i][0])
    
    x_samples.append(my_dict[i][0][k % n])
    y_samples.append(my_dict[i][1][k % n])

x_train = np.array(x_samples)
y_train = np.array(y_samples)

In [18]:
print(len(classes_list))
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

41
(21935, 7)
(21935, 4)
(90, 7)
(90, 4)


In [19]:
x_train.tofile('/content/drive/My Drive/Bases/DTP/x_train_22.dat')
y_train.tofile('/content/drive/My Drive/Bases/DTP/y_train_22.dat')

x_val.tofile('/content/drive/My Drive/Bases/DTP/x_val_22.dat')
y_val.tofile('/content/drive/My Drive/Bases/DTP/y_val_22.dat')
